<a href="https://colab.research.google.com/github/youmjisun/gachon_3-2/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#정형데이터, 비정형 데이터
- 정형데이터 : csv 파일 처럼, 구조를 가지고 있는 데이터
- 비정형데이터 : 덱스트 데이터, 사진, 디지털음악 등

#앙상블 학습
: 정형데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘

#랜덤 포레스트
: 앙상블학습 중 가장 유명하고 안정적인 학습을 제공
1. 훈련하기 위한 데이터를 랜덤하게 만듦
- 입력한 훈련 데이터에서 랜덤하게 샘플을 추출
- 중복된 샘플 추출가능
- 샘플을 부트스트랩 샘플이라 함
- 기본적으로 훈련세트의 크기와 동일하게 설정 됨
2. 부트스트랩 샘플로 결정트리를 훈련
- 각 로드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른다음 다음 이 중에서 최선의 분할을 찾음
-
-
3. 정해진 수 만큼 과정 반


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)


from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


#엑스트라 트리
: 랜덤 포레스트와 비슷하게 동작
: 부트스트랩 샘플을 사용하지 않고 각 결정 트리를 만들 때

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


#bagging
: 여러개의 모델을 랜덤하게

#boosting
: 모델을 순차적으로 여러개 만들되 이전 모델을 보완한 새로운 모델을 만들고 최종적으로 생성된 모든 모델을 하나로 합침

#그레디언트 부스팅
: 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식
: 과대적합에 강함 (깊이가 얕아서) = 일반화성이 높다

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb.fit(train_input, train_target)
print(gb.score(train_input, train_target))
print(gb.score(test_input, test_target))

0.8881086892152563 0.8720430147331015
0.8880123147969983
0.8669230769230769


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


#사스토그램 기반 그레디언트 부스팅
: 정형 데이터를 다루는 머신러닝 알고리즘 중에서 가장 인가가 높은 알고리

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

In [ ]:
#XGBoost
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [ ]:
#LightBoost
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


##실습
Wine 데이터 세트를 이용하여
- 결정트리
- 랜덤포레스트
- 엑스트라 트리
- 그레디언트 부스팅
- 히스토그램기반 그래디언트 부스팅
모델을 만들고 최적화

In [ ]:
#와인 데이터 불러오기
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
